In [ ]:
import collections
import hashlib
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import math
import random
from collections import Counter

In [ ]:
import pandas as pd
import os

create a dataframe object from the csv file

In [ ]:
new_dataframe = pd.read_csv(r"C:\Users\pushp\Downloads\Geolife-Trajectories-1.3\Geolife-Trajectories-1.3\Modified_Final_Dataset.csv")
new_dataframe.head()

Next few parts, we will be making our data in a usable format for feeding into our dataset

In [ ]:
def list_generate(new_dataframe,user_num):    
    user_lat = new_dataframe[new_dataframe.User_ID==user_num]['Modified_Latitude'].to_list()
    user_lon = new_dataframe[new_dataframe.User_ID==user_num]['Modified_Longitude'].to_list()
    user_timestamp = new_dataframe[new_dataframe.User_ID==user_num]['Timestamp'].to_list()
    l = len(new_dataframe[new_dataframe.user_num==57])
    user_path = [(querier_lat[i], querier_lon[i], querier_timestamp[i]) for i in range(l)]
    return user_path


For 50 infected users, we will create a list of lists where each list is the path of one infected patient

In [ ]:
querier_path = list_generate(new_dataframe,57)
infected_list = []
infected1 = list_generate(new_dataframe,94)
infected2 = list_generate(new_dataframe,150)
infected_list.append(infected1)
infected_list.append(infected2)
for i in range(50):
    infected_list.append(list_generate(new_dataframe,i))
    

In [ ]:
class Differential_Privacy_CMS():
    def __init__(self, width, depth,epsilon,seed):
        ''' Method to initialize the data structure
        @param width int: Width of the table
        @param depth int: Depth of the table (num of hash func)
        @param epsilon: privacy parameter
        In this case, when we declare differential privacy class instance, we give the parameters m and k, or width and depth
        '''
        self.width = width #hash values range between 0 to width-1
        self.depth = depth #number of hash functions
        self.epsilon = epsilon 
        self.seed = seed
    #Implementing the client side algorithm which works on each particular data element. First it selects an integer randomly from 0 to depth-1, 
    #we store that value as j, next, we initialize a vector of -1, of length width, and set that jth index as 1. Then we create another vector of -1s
    #and +1s, with fixed probabilities determined by the parameter epsilon. Then we do element wise multioplication and return the 
    #result vector. 
    def Client_Side(self,d):
        '''parameters are a data element, epsilon value and a hash family, but we are using mmh3 here so didn't add that as a parameter'''
        j = np.random.randint(self.depth) 
        z_vect = np.zeros((1,self.width))
        v = z_vect-1 
        hex_num = hashlib.sha256((str(self.seed[j])+str(d)).encode('utf-8')).hexdigest()
        int_hex = int(hex_num[:5],16)
        index = int_hex % self.width
        v[0,index]=1
        val = np.exp(self.epsilon/2)
        probability_of_1 = val/(val+1) 
        probability_of_neg_1 = 1/(val+1)
        b = np.random.choice([1,-1],self.width,p=[probability_of_1,probability_of_neg_1]) 
        final_vector = v*b
        return (final_vector,j) 
    


In [ ]:
class Querier(Differential_Privacy_CMS):
    def __init__(self, true_path,width, depth,epsilon, max,seed):
        super().__init__(width,depth,epsilon,max,seed)
        self.true_path = true_path
    def frequency_counter(self, Sketch_Matrix, total_path_length):
        ''' we get a sketch matrix and we try to get the frequencies for all elements in the true_path.'''
        frequency_dict = {}
        true_path = set(self.true_path)
        length = len(true_path)
        for d in self.true_path:
            n = length 
            frac1 = self.width/(self.width-1)
            frac2 = n/self.width
            row_sum = 0
            for i in range(self.depth):
                hex_num = hashlib.sha256((str(self.seed[i])+str(d)).encode('utf-8')).hexdigest()
                int_hex = int(hex_num[:5],16)
                index = int_hex % self.width
                row_sum = row_sum + Sketch_Matrix[i,index]
            avg_row_sum = row_sum/self.depth 
            subtraction = avg_row_sum - frac2 
            assumed_freq = frac1*subtraction 
            frequency_dict[d]=assumed_freq
        result_list = []
        for elt in frequency_dict:
            if frequency_dict[elt]>1:
                result_list.append(converter(elt))
        return result_list
        


In [ ]:
class Infected(Differential_Privacy_CMS):
    def __init__(self, true_path,width, depth,epsilon,max,seed):
        super().__init__(width,depth,epsilon,max,seed)
        self.true_path = true_path
    def send_flipped_bit_vector(self):
        result_list = []
        for data in self.true_path:
            result_list.append(self.Client_Side(data))
        return result_list



In [ ]:
class Server(Differential_Privacy_CMS):
    def __init__(self, width, depth, epsilon, max, seed):
        super().__init__(width,depth,epsilon,max,seed)
    def Compute_Sketch_Matrix(self,D):
        '''So each element of D is a tuple where the first element is v_i and second element is j_i(which is created by the above funstion after getting 
        passed by Client_side each time), we have privacy parameter epsilon and dimensions. v_i is the a vector an it has the sma eshape as the 
        vectors returned by the Client_side algorithm'''
        val = np.exp(self.epsilon/2)
        n = len(D)
        c_epsilon = (val+1)/(val-1) 
        vec_one = np.ones((1,self.width))
        manipulated_data_matrix = np.zeros((n,self.width)) #Creatimg a matrix for x_is
        for elt in enumerate(D):
            new_vect = elt[1][0].reshape((1,self.width))
            man_vect = (c_epsilon/2)*new_vect #scalar and vector multiplication 
            half_vec_one = 0.5*vec_one 
            sum_vect = man_vect + half_vec_one
            manipulated_data_matrix[[elt[0]],:]= self.depth*sum_vect  
        M = np.zeros((self.depth,self.width))
        for elt in enumerate(D):
            for l in range(self.width): 
                M[elt[1][1],l]=M[elt[1][1],l]+manipulated_data_matrix[elt[0],l]
        return M 

In [ ]:
#in the new mode, we invoke this method and the server receives this flipped bit vectors, as we want to average the 
#bit flipping thing, we run this process a mentioned number of times. and make compute sketch matrix.

In [ ]:
def driver_code(width, depth, seed, epsilon, run_number):
    '''1. Create instances of querier, server and infected patients.
       2. Call the filter method on all the participants in the server, so all the querier and the infectd patients filter out their paths 
          in multiple universes. This function will output a dictionary kepys mapping to 2 lists, one list is the co-ordinates in that universe and another list is the timestamp.
       3. Now send this filtered timestamp and zone list to the server. Form can be (zone code->timestamps in that zone)
       4. Taking the querier's timestamp against the infected patient's timestamp, server will level2 filtering and send similar form of 
          dictionary to all the infected patients and the querier, so basically finding the timestamp intersection between querier and infected patient pair
          and server is keeping the lists for his calculation.
       5. The participants, upon receiving the information, filters out the information further and start processing data from their end, aka, 
          calls the server caller method.(So in our implementation, we will be processing all infected patients one by one)
       6. The server caller method, is a common method which will be used in other parts too, when this method is called from some 
          infected patient, it asks for flipped vectors a mentioned number of times from the infected patient and upon receiving it every time,
          the server does the compute sketch matrix operation and contnues averaging it and finally gets the aggregated sketch matrix, as the server knows which 
          infected patient and from which zone, it was being called, he finally calls freq generator with this information.(Zone code, sketch matrix and server has saved the timestamp info
          for generating universe list)
       7. Thus for each caller method, server keeps generating frequency histogram and keeps aggregating it and will finally send it to the querier 
       
    '''
    

In [ ]:
s = driver_code(4.5)

In [ ]:
querier_path = [(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (0, 10), (0, 11), (0, 12), (0, 13), (0, 14), (0, 15), (0, 16), (0, 17), (0, 18), (0, 19), (0, 20), (0, 21), (0, 22), (0, 23), (0, 24), (0, 25), (0, 26), (0, 27), (0, 28), (0, 29), (0, 30), (0, 31), (0, 32), (0, 33), (0, 34), (0, 35), (0, 36), (0, 37), (0, 38), (0, 39), (0, 40), (0, 41), (0, 42), (0, 43), (0, 44), (0, 45), (0, 46), (0, 47), (0, 48), (0, 49), (0, 50), (0, 51), (0, 52), (0, 53), (0, 54), (0, 55), (0, 56), (0, 57), (0, 58), (0, 59), (0, 60), (1, 60), (2, 60), (3, 60), (4, 60), (5, 60), (6, 60), (7, 60), (8, 60), (9, 60), (10, 60), (11, 60), (12, 60), (13, 60), (14, 60), (15, 60), (16, 60), (17, 60), (18, 60), (19, 60), (20, 60), (21, 60), (22, 60), (23, 60), (24, 60), (25, 60), (26, 60), (27, 60), (28, 60), (29, 60), (30, 60), (31, 60), (32, 60), (33, 60), (34, 60), (35, 60), (36, 60), (37, 60), (38, 60), (39, 60), (40, 60), (41, 60), (42, 60), (43, 60), (44, 60), (45, 60), (46, 60), (47, 60), (48, 60), (49, 60), (50, 60), (51, 60), (52, 60), (53, 60), (54, 60), (55, 60), (56, 60), (57, 60), (58, 60), (59, 60), (60, 60), (61, 60), (62, 60), (63, 60), (64, 60), (65, 60), (66, 60), (67, 60), (68, 60), (69, 60), (70, 60), (70, 59), (70, 58), (70, 57), (70, 56), (70, 55), (70, 54), (70, 53), (70, 52), (70, 51), (70, 50), (70, 49), (70, 48), (70, 47), (70, 46), (70, 45), (70, 44), (70, 43), (70, 42), (70, 41), (70, 40), (70, 39), (70, 38), (70, 37), (70, 36), (70, 35), (70, 34), (70, 33), (70, 32), (70, 31), (70, 30), (70, 29), (70, 28), (70, 27), (70, 26), (70, 25), (70, 24), (70, 23), (70, 22), (70, 21), (70, 20), (69, 20), (68, 20), (67, 20), (66, 20), (65, 20), (64, 20), (63, 20), (62, 20), (61, 20), (60, 20), (59, 20), (58, 20), (57, 20), (56, 20), (55, 20), (54, 20), (53, 20), (52, 20), (51, 20), (50, 20), (49, 20), (48, 20), (47, 20), (46, 20), (45, 20), (44, 20), (43, 20), (42, 20), (41, 20), (40, 20), (39, 20), (38, 20), (37, 20), (36, 20), (35, 20), (34, 20), (33, 20), (32, 20), (31, 20), (30, 20)]
infected_true_path1=[(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (5, 0), (6, 0), (7, 0), (8, 0), (9, 0), (10, 0), (11, 0), (12, 0), (13, 0), (14, 0), (15, 0), (16, 0), (17, 0), (18, 0), (19, 0), (20, 0), (21, 0), (22, 0), (23, 0), (24, 0), (25, 0), (26, 0), (27, 0), (28, 0), (29, 0), (30, 0), (31, 0), (32, 0), (33, 0), (34, 0), (35, 0), (36, 0), (37, 0), (38, 0), (39, 0), (40, 0), (41, 0), (42, 0), (43, 0), (44, 0), (45, 0), (46, 0), (47, 0), (48, 0), (49, 0), (50, 0), (50, 1), (50, 2), (50, 3), (50, 4), (50, 5), (50, 6), (50, 7), (50, 8), (50, 9), (50, 10), (50, 11), (50, 12), (50, 13), (50, 14), (50, 15), (50, 16), (50, 17), (50, 18), (50, 19), (50, 20), (50, 21), (50, 22), (50, 23), (50, 24), (50, 25), (50, 26), (50, 27), (50, 28), (50, 29), (50, 30), (50, 31), (50, 32), (50, 33), (50, 34), (50, 35), (50, 36), (50, 37), (50, 38), (50, 39), (50, 40), (50, 41), (50, 42), (50, 43), (50, 44), (50, 45), (50, 46), (50, 47), (50, 48), (50, 49), (50, 50), (50, 51), (50, 52), (50, 53), (50, 54), (50, 55), (50, 56), (50, 57), (50, 58), (50, 59), (50, 60), (50, 61), (50, 62), (50, 63), (50, 64), (50, 65), (50, 66), (50, 67), (50, 68), (50, 69), (50, 70), (51, 70), (52, 70), (53, 70), (54, 70), (55, 70), (56, 70), (57, 70), (58, 70), (59, 70), (60, 70), (61, 70), (62, 70), (63, 70), (64, 70), (65, 70), (66, 70), (67, 70), (68, 70), (69, 70), (70, 70), (71, 70), (72, 70), (73, 70), (74, 70), (75, 70), (76, 70), (77, 70), (78, 70), (79, 70), (80, 70), (80, 69), (80, 68), (80, 67), (80, 66), (80, 65), (80, 64), (80, 63), (80, 62), (80, 61), (80, 60), (80, 59), (80, 58), (80, 57), (80, 56), (80, 55), (80, 54), (80, 53), (80, 52), (80, 51), (80, 50), (80, 49), (80, 48), (80, 47), (80, 46), (80, 45), (80, 44), (80, 43), (80, 42), (80, 41), (80, 40), (80, 39), (80, 38), (80, 37), (80, 36), (80, 35), (80, 34), (80, 33), (80, 32), (80, 31), (80, 30), (80, 29), (80, 28), (80, 27), (80, 26), (80, 25), (80, 24), (80, 23), (80, 22), (80, 21), (80, 20)]
infected_true_path2=[(0, 20), (1, 20), (2, 20), (3, 20), (4, 20), (5, 20), (6, 20), (7, 20), (8, 20), (9, 20), (10, 20), (11, 20), (12, 20), (13, 20), (14, 20), (15, 20), (16, 20), (17, 20), (18, 20), (19, 20), (20, 20), (21, 20), (22, 20), (23, 20), (24, 20), (25, 20), (26, 20), (27, 20), (28, 20), (29, 20), (30, 20), (30, 21), (30, 22), (30, 23), (30, 24), (30, 25), (30, 26), (30, 27), (30, 28), (30, 29), (30, 30), (30, 31), (30, 32), (30, 33), (30, 34), (30, 35), (30, 36), (30, 37), (30, 38), (30, 39), (30, 40), (30, 41), (30, 42), (30, 43), (30, 44), (30, 45), (30, 46), (30, 47), (30, 48), (30, 49), (30, 50), (30, 51), (30, 52), (30, 53), (30, 54), (30, 55), (30, 56), (30, 57), (30, 58), (30, 59), (30, 60), (30, 61), (30, 62), (30, 63), (30, 64), (30, 65), (30, 66), (30, 67), (30, 68), (30, 69), (30, 70), (30, 71), (30, 72), (30, 73), (30, 74), (30, 75), (30, 75), (31, 75), (32, 75), (33, 75), (34, 75), (35, 75), (36, 75), (37, 75), (38, 75), (39, 75), (40, 75), (41, 75), (42, 75), (43, 75), (44, 75), (45, 75), (46, 75), (47, 75), (48, 75), (49, 75), (50, 75), (51, 75), (52, 75), (53, 75), (54, 75), (55, 75), (56, 75), (57, 75), (58, 75), (59, 75), (60, 75), (60, 74), (60, 73), (60, 72), (60, 71), (60, 70), (60, 69), (60, 68), (60, 67), (60, 66), (60, 65), (60, 64), (60, 63), (60, 62), (60, 61), (60, 60), (60, 59), (60, 58), (60, 57), (60, 56), (60, 55), (60, 54), (60, 53), (60, 52), (60, 51), (60, 50), (60, 49), (60, 48), (60, 47), (60, 46), (60, 45), (60, 44), (60, 43), (60, 42), (60, 41), (60, 40), (60, 39), (60, 38), (60, 37), (60, 36), (60, 35), (60, 34), (60, 33), (60, 32), (60, 31), (60, 30), (60, 29), (60, 28), (60, 27), (60, 26), (60, 25), (60, 24), (60, 23), (60, 22), (60, 21), (60, 20), (60, 19), (60, 18), (60, 17), (60, 16), (60, 15), (61, 15), (62, 15), (63, 15), (64, 15), (65, 15), (66, 15), (67, 15), (68, 15), (69, 15), (70, 15), (71, 15), (72, 15), (73, 15), (74, 15), (75, 15), (76, 15), (77, 15), (78, 15), (79, 15), (80, 15), (80, 16), (80, 17), (80, 18), (80, 19), (80, 20), (80, 21), (80, 22), (80, 23), (80, 24), (80, 25), (80, 26), (80, 27), (80, 28), (80, 29), (80, 30), (80, 31), (80, 32), (80, 33), (80, 34), (80, 35), (80, 36), (80, 37), (80, 38), (80, 39), (80, 40), (80, 41), (80, 42), (80, 43), (80, 44), (80, 45), (80, 46), (80, 47), (80, 48), (80, 49), (80, 50), (80, 51), (80, 52), (80, 53), (80, 54), (80, 55), (80, 56), (80, 57), (80, 58), (80, 59), (80, 60), (80, 61), (80, 62), (80, 63), (80, 64), (80, 65), (80, 66), (80, 67), (80, 68), (80, 69), (80, 70)]
infected_true_path3=[(0, 50), (1, 50), (2, 50), (3, 50), (4, 50), (5, 50), (6, 50), (7, 50), (8, 50), (9, 50), (10, 50), (11, 50), (12, 50), (13, 50), (14, 50), (15, 50), (16, 50), (17, 50), (18, 50), (19, 50), (20, 50), (20, 51), (20, 52), (20, 53), (20, 54), (20, 55), (20, 56), (20, 57), (20, 58), (20, 59), (20, 60), (20, 61), (20, 62), (20, 63), (20, 64), (20, 65), (20, 66), (20, 67), (20, 68), (20, 69), (20, 70), (20, 71), (20, 72), (20, 73), (20, 74), (20, 75), (21, 75), (22, 75), (23, 75), (24, 75), (25, 75), (26, 75), (27, 75), (28, 75), (29, 75), (30, 75), (31, 75), (32, 75), (33, 75), (34, 75), (35, 75), (36, 75), (37, 75), (38, 75), (39, 75), (40, 75), (41, 75), (42, 75), (43, 75), (44, 75), (45, 75), (46, 75), (47, 75), (48, 75), (49, 75), (50, 75), (50, 74), (50, 73), (50, 72), (50, 71), (50, 70), (50, 69), (50, 68), (50, 67), (50, 66), (50, 65), (50, 64), (50, 63), (50, 62), (50, 61), (50, 60), (50, 59), (50, 58), (50, 57), (50, 56), (50, 55), (50, 54), (50, 53), (50, 52), (50, 51), (50, 50), (50, 49), (50, 48), (50, 47), (50, 46), (50, 45), (50, 44), (50, 43), (50, 42), (50, 41), (50, 40), (50, 39), (50, 38), (50, 37), (50, 36), (50, 35), (49, 35), (48, 35), (47, 35), (46, 35), (45, 35), (44, 35), (43, 35), (42, 35), (41, 35), (40, 35), (39, 35), (38, 35), (37, 35), (36, 35), (35, 35), (34, 35), (33, 35), (32, 35), (31, 35), (30, 35), (29, 35), (28, 35), (27, 35), (26, 35), (25, 35), (24, 35), (23, 35), (22, 35), (21, 35), (20, 35), (20, 34), (20, 33), (20, 32), (20, 31), (20, 30), (20, 29), (20, 28), (20, 27), (20, 26), (20, 25), (20, 24), (20, 23), (20, 22), (20, 21), (20, 20), (20, 19), (20, 18), (20, 17), (20, 16), (20, 15), (21, 15), (22, 15), (23, 15), (24, 15), (25, 15), (26, 15), (27, 15), (28, 15), (29, 15), (30, 15), (31, 15), (32, 15), (33, 15), (34, 15), (35, 15), (36, 15), (37, 15), (38, 15), (39, 15), (40, 15), (41, 15), (42, 15), (43, 15), (44, 15), (45, 15), (46, 15), (47, 15), (48, 15), (49, 15), (50, 15), (51, 15), (52, 15), (53, 15), (54, 15), (55, 15), (56, 15), (57, 15), (58, 15), (59, 15), (60, 15), (61, 15), (62, 15), (63, 15), (64, 15), (65, 15), (66, 15), (67, 15), (68, 15), (69, 15), (70, 15), (71, 15), (72, 15), (73, 15), (74, 15), (75, 15), (76, 15), (77, 15), (78, 15), (79, 15), (80, 15)]

In [ ]:
original_intersection = set(querier_path) & set(infected_true_path1+infected_true_path2+infected_true_path3)

In [ ]:
len(original_intersection)

10

In [ ]:
set(s) & original_intersection

{(0, 50), (20, 60), (30, 20), (30, 60), (50, 20), (50, 60), (60, 20)}

In [ ]:
len(set(s))

103

In [ ]:
l = {(17, 60), (23, 60), (61, 20), (0, 56), (66, 20), (70, 30), (0, 43), (70, 23), (57, 20), (40, 60), (63, 20), (1, 60), (37, 60), (70, 48), (34, 60), (32, 20), (21, 60), (52, 20), (47, 60), (5, 60), (54, 20), (53, 20), (0, 22), (59, 20), (58, 20), (64, 20), (0, 25), (38, 60), (28, 60), (0, 45), (10, 60), (40, 20), (0, 12), (15, 60), (0, 38), (35, 60), (0, 30), (22, 60), (70, 47), (42, 60), (65, 20), (32, 60), (70, 36), (19, 60), (62, 20), (67, 20), (55, 20), (62, 60), (33, 60), (70, 22), (69, 60), (36, 20), (51, 20), (0, 2), (56, 20)}